In [38]:
#!pip install pypdf
#!pip install chromadb
#!pip install gpt4all
#!pip install sentence-transformers

  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00
  Using cached pillow-10.3.0-cp39-cp39-macosx_10_10_x86_64.whl.metadata (9.2 kB)
  Using cached safetensors-0.4.3-cp39-cp39-macosx_10_12_x86_64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 4.8 MB/s eta 0:00:0000:0100:01
Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)
Using cached pillow-10.3.0-cp39-cp39-macosx_10_10_x86_64.whl (3.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 7.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 7.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 3.1 MB/s eta 0:00:00:00:01
Using cached safetensors-0.4.3-cp39-cp39-macosx_10_12_x86_64.whl (416 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings

from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from gpt4all import Embed4All
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

import getpass

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")

_set_if_undefined("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="AI Assistant"


CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [5]:
def get_embedding_function():
    #embeddings = BedrockEmbeddings(credentials_profile_name="default", region_name="us-east-1")
    #embeddings = OllamaEmbeddings(model="nomic-embed-text")
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return embeddings

In [6]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [7]:
doc = load_documents()

In [8]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [9]:
chunks = split_documents(doc)

In [10]:
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


In [11]:
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

In [12]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

In [13]:
clear_database()

In [14]:
add_to_chroma(chunks)

/Users/sergekeita/opt/anaconda3/envs/llm_langgraph/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/sergekeita/opt/anaconda3/envs/llm_langgraph/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/sergekeita/opt/anaconda3/envs/llm_langgraph/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132:

Number of existing documents in DB: 0
👉 Adding new documents: 4


/Users/sergekeita/opt/anaconda3/envs/llm_langgraph/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [15]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [16]:
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="openhermes")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [17]:
question = "Serge à t il des compétence en python et AWS ?"

In [19]:
query_rag(question)

/Users/sergekeita/opt/anaconda3/envs/llm_langgraph/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Response: Oui, Serge a des compétences en Python et AWS. Il est mentionné qu'il a travaillé comme Développeur Python & Rust à LEDR Technologies et est certifié AWS Solutions Architect Associate. Il possède également des expériences professionnelles impliquant le déploiement de solutions informatiques sur AWS et l'utilisation d'outils tels que Terraform pour la conception d'architectures AWS.
Sources: ['data/CV Serge Keita.pdf:0:0', 'data/CV Serge Keita.pdf:0:1', 'data/CV Serge Keita.pdf:0:2', 'data/CV Serge Keita.pdf:0:3']


"Oui, Serge a des compétences en Python et AWS. Il est mentionné qu'il a travaillé comme Développeur Python & Rust à LEDR Technologies et est certifié AWS Solutions Architect Associate. Il possède également des expériences professionnelles impliquant le déploiement de solutions informatiques sur AWS et l'utilisation d'outils tels que Terraform pour la conception d'architectures AWS."

In [20]:
from transformers import pipeline

In [23]:
def query_rag2(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    #pipeline("text-generation", model="tiiuae/falcon-7b-instruct", trust_remote_code=True)
    llm = pipeline("text-generation", model="gpt2", framework="pt")  # force PyTorch framework

    #model = Ollama(model="openhermes")
    #response_text = model.invoke(prompt)
    response = llm(prompt, max_length=1000)
    response_text = response[0]["generated_text"]

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [24]:
query_rag2(question)

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: Human: 
Answer the question based only on the following context:

COMPÉTENCESS E R G E
KEITAData Enthousiaste | Machine Learning | Devops
Cloud | Python Développeur
POINTS FORTS+33698480507 @  sergekeita01@gmail.com 
CERTIFICATIONS
AWS Cloud Quest : Machine
Learning
AWS Solutions Architect Associate 
HarshiCorp Terraform Associate 003
Certification CISCO CCNA1 
Certification AZ 900 (2022)
TOEICData Scientest
English (Fluent)
French (Fluent)LANGUAGESEXPERIENCES
FORMATIONSPROFILE
Fort d'une expérience internationale et d’un master en Big Data & Artificial
Intelligence, j'ai acquis de solides connaissances en programmation, machine
learning et le déploiement de solutions informatiques sur AWS. Passionné par
la data, j'aime explorer de nouvelles idées et proposer des solutions créatives
pour relever les défis du domaine.

---

pour relever les défis du domaine.
12/2023 - 04/2024 Consultant Devops Cloud AWS
Data Scientest X Storm Reply - POEI
02/2023 - 08/2023 Consultant Data & Cy

"Human: \nAnswer the question based only on the following context:\n\nCOMPÉTENCESS E R G E\nKEITAData Enthousiaste | Machine Learning | Devops\nCloud | Python Développeur\nPOINTS FORTS+33698480507 @  sergekeita01@gmail.com \nCERTIFICATIONS\nAWS Cloud Quest : Machine\nLearning\nAWS Solutions Architect Associate \nHarshiCorp Terraform Associate 003\nCertification CISCO CCNA1 \nCertification AZ 900 (2022)\nTOEICData Scientest\nEnglish (Fluent)\nFrench (Fluent)LANGUAGESEXPERIENCES\nFORMATIONSPROFILE\nFort d'une expérience internationale et d’un master en Big Data & Artificial\nIntelligence, j'ai acquis de solides connaissances en programmation, machine\nlearning et le déploiement de solutions informatiques sur AWS. Passionné par\nla data, j'aime explorer de nouvelles idées et proposer des solutions créatives\npour relever les défis du domaine.\n\n---\n\npour relever les défis du domaine.\n12/2023 - 04/2024 Consultant Devops Cloud AWS\nData Scientest X Storm Reply - POEI\n02/2023 - 08/2023 